Dans ce notebook, nous allons essayer de faire du text mining pour récuper des versions locales des programmes des présidentielles 2017 des candidats suivants :

- François Fillon (apparemment, le projet sort le 13 mars seulement)
- Marine Le Pen
- Benoît Hamon
- Jean-Luc Mélenchon
- Emmanuel Macron

Différentes manières de récupérer les programmes sont possibles : soit à partir des fichiers .pdf, soit à partir des sites des campagnes. Nous allons faire au plus simple, à l'aide des sites de campagnes.

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from ipywidgets import interact

In [40]:
def make_df_from_props_sources(props_sources):
    "Makes a big dataframe from props_sources."
    dfs = []
    for key in props_sources:
        df = pd.DataFrame(props_sources[key], columns=['proposition'])
        df['source'] = key
        dfs.append(df)
    df = pd.concat(dfs).reset_index(drop=True)
    return df

# François Fillon 

Le projet de François Fillon ne sera annoncé que le 13 mars : https://www.fillon2017.fr/projet/

In [2]:
r = requests.get('https://www.fillon2017.fr/projet/')
soup = BeautifulSoup(r.text, 'html.parser')

In [3]:
tags = soup.find_all('a', class_='projectItem__inner')

In [4]:
sublinks = [tag.attrs['href'] for tag in tags]

In [6]:
r = requests.get('https://www.fillon2017.fr/projet/competitivite/')
soup = BeautifulSoup(r.text, 'html.parser')

In [10]:
tags = soup.find_all('li', class_='singleProject__propositionItem')

In [20]:
len(tags)

5

In [13]:
tag = tags[0]

In [24]:
tag.find('div', class_='singleProject__propositionItem-content').text

'Prolonger jusqu’en 2019 le dispositif de suramortissement exceptionnel des investissements, pour soutenir l’investissement et la trésorerie des entreprises industrielles. \n'

In [25]:
for tag in tags:
    tag.find('div', class_='singleProject__propositionItem-content').text

In [26]:
def extract_propositions(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    tags = soup.find_all('li', class_='singleProject__propositionItem')
    return [tag.find('div', class_='singleProject__propositionItem-content').text for tag in tags]

In [27]:
extract_propositions(sublinks[0])

['Renforcer les allègements existants par un nouvel allègement de charges sociales et d’impôts de production, de 25 Mds€, portant sur tous les salaires, pour redonner un nouveau souffle à l’économie française. Cet allègement montera progressivement en charge jusqu’en 2020. ',
 'Transformer le CICE qui est un crédit d’impôt complexe, en baisse des charges patronales pour un montant équivalent, durable et lisible.',
 "Alléger l’impôt sur les sociétés (IS) pour renforcer la compétitivité de nos entreprises et la création d’emplois en France. Les contributions additionnelles à l’IS seront supprimées en 2018 et le taux d'IS sera diminué progressivement pour atteindre environ 25% au terme du quinquennat. Cette mesure représente un effort de l’ordre de 10 Mds€ et permettra de tendre vers le taux moyen de l’Union européenne.  \n",
 'Prolonger jusqu’en 2019 le dispositif de suramortissement exceptionnel des investissements, pour soutenir l’investissement et la trésorerie des entreprises industr

In [28]:
props_sources = {}
for sublink in sublinks:
    props = extract_propositions(sublink)
    props_sources[sublink] = props

In [93]:
df = make_df_from_props_sources(props_sources)
df.head()

,proposition,source
0,Inscrire dans la Constitution le principe de q...,https://www.fillon2017.fr/projet/immigration/
1,Développer l'usage des statistiques d'origine ...,https://www.fillon2017.fr/projet/immigration/
2,Durcir les exigences du regroupement familial ...,https://www.fillon2017.fr/projet/immigration/
3,Restaurer notre souveraineté migratoire en ren...,https://www.fillon2017.fr/projet/immigration/
4,Abroger les récentes réformes législatives éla...,https://www.fillon2017.fr/projet/immigration/


In [94]:
df.to_csv('../projets/francois_fillon.csv', index=False)

# Marine Le Pen 

Les 144 engagements de Marine Le Pen peuvent être consultés ici : https://www.marine2017.fr/programme/

## Analyse de la structure du site

Apparemment, les différentes propositions sont imbriquées dans des balises `<p>`. 

```
<p>3. <strong>Permettre la représentation de tous les Français</strong> par le scrutin proportionnel à toutes les élections. À l’Assemblée nationale, la proportionnelle sera intégrale avec une prime majoritaire de 30&nbsp;% des sièges pour la liste arrivée en tête et un seuil de 5&nbsp;% des suffrages pour obtenir des élus.</p>
```
On peut donc extraire ces éléments et les trier ensuite.

## Extraction des paragraphes

Téléchargeons le code source de la page.

In [98]:
r = requests.get('https://www.marine2017.fr/programme/')

In [99]:
soup = BeautifulSoup(r.text, "html.parser")

Maintenant, chercons à extraire tous les paragraphes, à l'aide d'une fonction qui vérifie que le paragraphe commence par un nombre suivi d'un point (et peut-être d'un espace).

In [122]:
pattern = re.compile('^\d+.\s*')

In [123]:
def filter_func(tag):    
    if tag.text is not None:
        return pattern.match(tag.text) is not None
    else:
        return False

In [124]:
all_paragraphs = [re.split(pattern, tag.text)[1:] for tag in soup.find_all('p') if filter_func(tag)]

In [125]:
len(all_paragraphs)

144

In [132]:
@interact
def disp_para(n=(0, len(all_paragraphs) - 1)):
    print(all_paragraphs[n])

In [133]:
props_sources = {}
props_sources['https://www.marine2017.fr/programme/'] = all_paragraphs

In [134]:
df = make_df_from_props_sources(props_sources)

In [135]:
df.head(10)

,proposition,source
0,Retrouver notre liberté et la maîtrise de notr...,https://www.marine2017.fr/programme/
1,Organiser un référendum en vue de réviser la C...,https://www.marine2017.fr/programme/
2,Permettre la représentation de tous les França...,https://www.marine2017.fr/programme/
3,Abaisser le nombre de députés à 300 (contre 57...,https://www.marine2017.fr/programme/
4,Créer un véritable référendum d’initiative pop...,https://www.marine2017.fr/programme/
5,Conserver trois niveaux d’administration (au l...,https://www.marine2017.fr/programme/
6,Garantir la liberté d’expression et les libert...,https://www.marine2017.fr/programme/
7,Créer une charte à valeur constitutionnelle qu...,https://www.marine2017.fr/programme/
8,Défendre les droits des femmes : lutter contre...,https://www.marine2017.fr/programme/
9,Assurer le respect de la liberté d’association...,https://www.marine2017.fr/programme/


Bien, on peut maintenant écrire ces données dans un fichier texte.

In [136]:
df.to_csv('../projets/marine_le_pen.csv', index=False)

# Benoît Hamon 

Le site de Benoît Hamon ne permet pas d'accéder à une page avec toutes les propositions facilement. Du coup, il faut explorer trois sous-catégories.

https://www.benoithamon2017.fr/thematique/pour-un-progres-social-et-ecologique/

In [145]:
r = requests.get('https://www.benoithamon2017.fr/thematique/pour-un-progres-social-et-ecologique/')
r

<Response [200]>

In [146]:
soup = BeautifulSoup(r.text, 'html.parser')

In [147]:
all_propositions = soup.find_all(class_='Propositions-Proposition')

In [148]:
len(all_propositions)

66

In [149]:
p = all_propositions[0]

In [150]:
p.text

'\nLutte contre les déserts médicaux\nJe lutterai contre les déserts médicaux en retirant le conventionnement aux médecins qui s’installent en zone surdotée. Cette mesure n’entrave en rien la liberté d’installation du médecin, qui peut toujours choisir de s’y installer\xa0: il ne bénéficiera simplement plus du conventionnement. J’encouragerai le développement de maisons de santé pluridisciplinaires rassemblant des médecins libéraux, appuyés pour les tâches administratives par un gestionnaire des fonctions support. \n\n\nJe soutiens, je partage sur les réseaux #DésertsMédicaux\n\n\n\nEn savoir plus\n'

In [151]:
p.find('h1').text

'Lutte contre les déserts médicaux'

In [152]:
p.find('p').text

'Je lutterai contre les déserts médicaux en retirant le conventionnement aux médecins qui s’installent en zone surdotée. Cette mesure n’entrave en rien la liberté d’installation du médecin, qui peut toujours choisir de s’y installer\xa0: il ne bénéficiera simplement plus du conventionnement. J’encouragerai le développement de maisons de santé pluridisciplinaires rassemblant des médecins libéraux, appuyés pour les tâches administratives par un gestionnaire des fonctions support.'

On peut extraire de ces propositions la moëlle essentielle :

In [154]:
def extract_data(tag):
    "Extracts title for tag and content."
    subject = tag.find('h1').text
    content = tag.find('p').text
    return subject, content

Construisons une table de données avec ces propositions.

In [155]:
df = pd.DataFrame([extract_data(p) for p in all_propositions], columns=['titre', 'contenu'])

In [156]:
df

,titre,contenu
0,Lutte contre les déserts médicaux,Je lutterai contre les déserts médicaux en ret...
1,Abrogation de la loi Travail,J’abrogerai immédiatement la loi Travail. Je r...
2,Un budget de 1% du PIB dédié à la culture,Je porterai le budget consacré au développemen...
3,Création d’un Revenu universel d’existence,Je mettrai en place un revenu universel d’exis...
4,Mise en place d’un statut social unique de l’a...,Je créerai un statut unique pour tous les acti...
5,"Oui au travail indépendant, non à l’ubérisatio...",Je lutterai contre le salariat déguisé des ent...
6,Plan d’investissements dans la rénovation éner...,Je lancerai un plan massif d’investissements d...
7,Annulation de la dette contractée par les pays...,J’organiserai avec nos partenaires européens l...
8,Création d’une taxe sur les robots,Je créerai une taxe sur la richesse créée par ...
9,Reconnaissance d’un statut de l’artiste,Je donnerai corps à un statut de l’artiste afi...


In [169]:
df[df['contenu'].str.contains('ascension')]

,titre,contenu
65,L’Université au coeur de la méritocratie répub...,Je remettrai l’enseignement supérieur au coeur...


On peut transformer ces propositions en DataFrame.

In [171]:
props_sources = {}
props_sources['https://www.benoithamon2017.fr/thematique/pour-un-progres-social-et-ecologique/'] = df['contenu'].values.tolist()

In [172]:
df = make_df_from_props_sources(props_sources)

In [174]:
df.head()

,proposition,source
0,Je lutterai contre les déserts médicaux en ret...,https://www.benoithamon2017.fr/thematique/pour...
1,J’abrogerai immédiatement la loi Travail. Je r...,https://www.benoithamon2017.fr/thematique/pour...
2,Je porterai le budget consacré au développemen...,https://www.benoithamon2017.fr/thematique/pour...
3,Je mettrai en place un revenu universel d’exis...,https://www.benoithamon2017.fr/thematique/pour...
4,Je créerai un statut unique pour tous les acti...,https://www.benoithamon2017.fr/thematique/pour...


In [175]:
df.to_csv('../projets/benoit_hamon.csv', index=False)

# Jean-Luc Mélenchon 

On peut trouver une version inofficielle du programme ici : https://laec.fr/sommaire

Un peu comme pour le site d'Hamon, il y a des rubriques. Commençons par la première.

In [176]:
r = requests.get('https://laec.fr/chapitre/1/la-6e-republique')

In [177]:
soup = BeautifulSoup(r.text, 'html.parser')

In [178]:
sublinks = soup.find_all('a', class_='list-group-item')

In [179]:
sublinks

[<a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/1/reunir-une-assemblee-constituante"><span class="label label-default">1</span> Réunir une Assemblée constituante </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/2/balayer-l-oligarchie-abolir-les-privileges-de-la-caste"><span class="label label-default">2</span> Balayer l'oligarchie, abolir les privilèges de la caste </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/3/une-republique-permettant-l-intervention-populaire"><span class="label label-default">3</span> Une République permettant l'intervention populaire </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/4/abolir-la-monarchie-presidentielle"><span class="label label-default">4</span> Abolir la monarchie présidentielle </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/5/une-nouvelle-etape-

On peut étendre cette manière de récupérer les données à toutes les sous-sections :

In [180]:
suburls = ['https://laec.fr/chapitre/1/la-6e-republique', 
           'https://laec.fr/chapitre/2/proteger-et-partager',
           'https://laec.fr/chapitre/3/la-planification-ecologique',
           'https://laec.fr/chapitre/4/sortir-des-traites-europeens',
           'https://laec.fr/chapitre/5/pour-l-independance-de-la-france',
           'https://laec.fr/chapitre/6/le-progres-humain-d-abord',
           'https://laec.fr/chapitre/7/la-france-aux-frontieres-de-l-humanite']

In [181]:
sublinks = []
for suburl in suburls:
    r = requests.get(suburl)
    soup = BeautifulSoup(r.text, 'html.parser')
    sublinks.extend(soup.find_all('a', class_='list-group-item'))

In [211]:
sublinks[:5]

[<a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/1/reunir-une-assemblee-constituante"><span class="label label-default">1</span> Réunir une Assemblée constituante </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/2/balayer-l-oligarchie-abolir-les-privileges-de-la-caste"><span class="label label-default">2</span> Balayer l'oligarchie, abolir les privilèges de la caste </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/3/une-republique-permettant-l-intervention-populaire"><span class="label label-default">3</span> Une République permettant l'intervention populaire </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/4/abolir-la-monarchie-presidentielle"><span class="label label-default">4</span> Abolir la monarchie présidentielle </a>,
 <a class="list-group-item toc-section toc-chapter-la-6e-republique" href="/section/5/une-nouvelle-etape-

Combien de propositions trouvons-nous ?

In [212]:
len(sublinks)

83

Construisons les url complètes.

In [216]:
full_urls = ['https://laec.fr' + link.attrs['href'] for link in sublinks]
full_urls[:10]

['https://laec.fr/section/1/reunir-une-assemblee-constituante',
 'https://laec.fr/section/2/balayer-l-oligarchie-abolir-les-privileges-de-la-caste',
 'https://laec.fr/section/3/une-republique-permettant-l-intervention-populaire',
 'https://laec.fr/section/4/abolir-la-monarchie-presidentielle',
 'https://laec.fr/section/5/une-nouvelle-etape-des-libertes-et-de-l-emancipation-personnelles',
 'https://laec.fr/section/6/une-republique-universelle',
 'https://laec.fr/section/7/une-republique-laique',
 'https://laec.fr/section/8/la-revolution-citoyenne-dans-les-medias',
 'https://laec.fr/section/9/la-republique-garante-des-biens-communs',
 'https://laec.fr/section/10/reconnaitre-la-citoyennete-dans-l-entreprise-et-des-droits-nouveaux-aux-salaries']

In [274]:
full_url = full_urls[13]
#full_url = full_urls[0]

In [275]:
r = requests.get(full_url)

In [276]:
print(r.text[:800])

<!DOCTYPE html>
<html lang="fr" class="page no-js not-logged-in theme-frontend theme-chapter-la-6e-republique page-section page-section-14">
<!--


░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
░░░█░░░█▀█░█▀▀░█▀▀░░░░█▀▀░█▀▄░░░░
░░░█░░░█▀█░█▀▀░█░░░░░░█▀▀░█▀▄░░░░
░░░▀▀▀░▀░▀░▀▀▀░▀▀▀░▀░░▀░░░▀░▀░░░░
░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
+===== L'Avenir En Commun ======+

-->
  <head>
    <!-- Classe .js/.no-js https://www.paulirish.com/2009/avoiding-the-fouc-v3/ -->
    <script>(function(H){H.className=H.className.replace(/\bno-js\b/,'js')})(document.documentElement)</script>
    
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <title>La jeunesse au service de l&#039;intérêt général et de la sûreté de la Nation</title>
    <meta name="d


In [277]:
soup = BeautifulSoup(r.text, 'html.parser')

In [278]:
tags = soup.find_all('li', class_='list-group-item')

In [279]:
tag = tags[0]

In [281]:
tag.text

"Créer un service citoyen obligatoirePour les femmes et les hommesPar conscription avant 25 ans, proche du lieu de vie, en limitant le casernement aux fonctions qui l'exigent réellementD'une durée totale de neuf mois, comprenant une formation militaire initiale incluant un droit à l'objection de conscienceRémunéré au smicAffecté à des tâches d'intérêt général\xa0: secours à la population, sapeurs-pompiers, sécurité publique, défense, sécurité civile, protection et réparation de l'environnement, appui à des associations labellisées d'intérêt généralPrésence sur tout le territoire, y compris les Outre-mer, les zones rurales et les quartiers populairesComprenant un bilan de santé, une évaluation des capacités d'écriture, de lecture et de calcul avec leur éventuelle mise à niveau, la formation gratuite à la conduite et le passage de l'examen du permis de conduire"

In [282]:
tag.find_all('li')

[<li>Pour les femmes et les hommes</li>,
 <li>Par conscription avant 25 ans, proche du lieu de vie, en limitant le <em>casernement</em> aux fonctions qui l'exigent réellement</li>,
 <li>D'une durée totale de neuf mois, comprenant une formation militaire initiale incluant un droit à l'objection de conscience</li>,
 <li>Rémunéré au <abbr title="Salaire Minimum Interprofessionnel de Croissance">smic</abbr></li>,
 <li>Affecté à des tâches d'intérêt général : secours à la population, sapeurs-pompiers, sécurité publique, défense, sécurité civile, protection et réparation de l'environnement, appui à des associations labellisées d'intérêt général</li>,
 <li>Présence sur tout le territoire, y compris les Outre-mer, les zones rurales et les quartiers populaires</li>,
 <li>Comprenant un bilan de santé, une évaluation des capacités d'écriture, de lecture et de calcul avec leur éventuelle mise à niveau, la formation gratuite à la conduite et le passage de l'examen du permis de conduire</li>]

In [288]:
tag.p.text

'Créer un service citoyen obligatoire'

In [284]:
"\n".join([t.text for t in tag.find_all('li')])

"Pour les femmes et les hommes\nPar conscription avant 25 ans, proche du lieu de vie, en limitant le casernement aux fonctions qui l'exigent réellement\nD'une durée totale de neuf mois, comprenant une formation militaire initiale incluant un droit à l'objection de conscience\nRémunéré au smic\nAffecté à des tâches d'intérêt général\xa0: secours à la population, sapeurs-pompiers, sécurité publique, défense, sécurité civile, protection et réparation de l'environnement, appui à des associations labellisées d'intérêt général\nPrésence sur tout le territoire, y compris les Outre-mer, les zones rurales et les quartiers populaires\nComprenant un bilan de santé, une évaluation des capacités d'écriture, de lecture et de calcul avec leur éventuelle mise à niveau, la formation gratuite à la conduite et le passage de l'examen du permis de conduire"

In [286]:
len(tags)

2

In [285]:
[tag.text for tag in tags]

["Créer un service citoyen obligatoirePour les femmes et les hommesPar conscription avant 25 ans, proche du lieu de vie, en limitant le casernement aux fonctions qui l'exigent réellementD'une durée totale de neuf mois, comprenant une formation militaire initiale incluant un droit à l'objection de conscienceRémunéré au smicAffecté à des tâches d'intérêt général\xa0: secours à la population, sapeurs-pompiers, sécurité publique, défense, sécurité civile, protection et réparation de l'environnement, appui à des associations labellisées d'intérêt généralPrésence sur tout le territoire, y compris les Outre-mer, les zones rurales et les quartiers populairesComprenant un bilan de santé, une évaluation des capacités d'écriture, de lecture et de calcul avec leur éventuelle mise à niveau, la formation gratuite à la conduite et le passage de l'examen du permis de conduire",
 "Créer une garde nationale placée sous commandement civil et composéeDes jeunes en service citoyen obligatoire ayant choisi 

In [294]:
def extract_data(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    tags = soup.find_all('li', class_='list-group-item')
    contents = []
    for tag in tags:
        if len(tag.find_all('li')) == 0:
            contents.append(tag.text)
        else:
            contents.append(tag.p.text + '\n\t' + "\n\t".join([t.text for t in tag.find_all('li')]))
    return contents

In [295]:
extract_data(full_url)

["Créer un service citoyen obligatoire\n\tPour les femmes et les hommes\n\tPar conscription avant 25 ans, proche du lieu de vie, en limitant le casernement aux fonctions qui l'exigent réellement\n\tD'une durée totale de neuf mois, comprenant une formation militaire initiale incluant un droit à l'objection de conscience\n\tRémunéré au smic\n\tAffecté à des tâches d'intérêt général\xa0: secours à la population, sapeurs-pompiers, sécurité publique, défense, sécurité civile, protection et réparation de l'environnement, appui à des associations labellisées d'intérêt général\n\tPrésence sur tout le territoire, y compris les Outre-mer, les zones rurales et les quartiers populaires\n\tComprenant un bilan de santé, une évaluation des capacités d'écriture, de lecture et de calcul avec leur éventuelle mise à niveau, la formation gratuite à la conduite et le passage de l'examen du permis de conduire",
 "Créer une garde nationale placée sous commandement civil et composée\n\tDes jeunes en service c

In [296]:
extract_data(full_urls[13])

["Créer un service citoyen obligatoire\n\tPour les femmes et les hommes\n\tPar conscription avant 25 ans, proche du lieu de vie, en limitant le casernement aux fonctions qui l'exigent réellement\n\tD'une durée totale de neuf mois, comprenant une formation militaire initiale incluant un droit à l'objection de conscience\n\tRémunéré au smic\n\tAffecté à des tâches d'intérêt général\xa0: secours à la population, sapeurs-pompiers, sécurité publique, défense, sécurité civile, protection et réparation de l'environnement, appui à des associations labellisées d'intérêt général\n\tPrésence sur tout le territoire, y compris les Outre-mer, les zones rurales et les quartiers populaires\n\tComprenant un bilan de santé, une évaluation des capacités d'écriture, de lecture et de calcul avec leur éventuelle mise à niveau, la formation gratuite à la conduite et le passage de l'examen du permis de conduire",
 "Créer une garde nationale placée sous commandement civil et composée\n\tDes jeunes en service c

In [297]:
print(_291[0])

Créer un service citoyen obligatoire
Pour les femmes et les hommes
Par conscription avant 25 ans, proche du lieu de vie, en limitant le casernement aux fonctions qui l'exigent réellement
D'une durée totale de neuf mois, comprenant une formation militaire initiale incluant un droit à l'objection de conscience
Rémunéré au smic
Affecté à des tâches d'intérêt général : secours à la population, sapeurs-pompiers, sécurité publique, défense, sécurité civile, protection et réparation de l'environnement, appui à des associations labellisées d'intérêt général
Présence sur tout le territoire, y compris les Outre-mer, les zones rurales et les quartiers populaires
Comprenant un bilan de santé, une évaluation des capacités d'écriture, de lecture et de calcul avec leur éventuelle mise à niveau, la formation gratuite à la conduite et le passage de l'examen du permis de conduire


In [298]:
props_sources = {}
for url in full_urls:
    props_sources[url] = extract_data(url)

In [299]:
df = make_df_from_props_sources(props_sources)
df

,proposition,source
0,Créer un service citoyen obligatoire\n\tPour l...,https://laec.fr/section/14/la-jeunesse-au-serv...
1,Créer une garde nationale placée sous commande...,https://laec.fr/section/14/la-jeunesse-au-serv...
2,Établir un diagnostic global des consommations...,https://laec.fr/section/69/changer-de-logique-...
3,"Légaliser et encadrer la consommation, la prod...",https://laec.fr/section/69/changer-de-logique-...
4,Affecter les recettes des taxes sur le cannabi...,https://laec.fr/section/69/changer-de-logique-...
5,Déclarer la souffrance au travail grande cause...,https://laec.fr/section/70/en-finir-avec-la-so...
6,Renforcer la médecine du travail\n\tIntégrer l...,https://laec.fr/section/70/en-finir-avec-la-so...
7,Reconnaître le burn-out comme maladie professi...,https://laec.fr/section/70/en-finir-avec-la-so...
8,Faire du nombre d'accidents du travail un crit...,https://laec.fr/section/70/en-finir-avec-la-so...
9,S'exonérer du pacte de stabilité et des règles...,https://laec.fr/section/49/prendre-les-mesures...


On écrit un fichier.

In [300]:
df.to_csv('../projets/jean_luc_melenchon.csv', index=False)

# Emmanuel Macron 

Il faut dans un premier temps aller chercher les pages individuelles du site.

In [301]:
r = requests.get('https://en-marche.fr/emmanuel-macron/le-programme')
soup = BeautifulSoup(r.text, 'html.parser')

In [302]:
proposals = soup.find_all(class_='programme__proposal')
proposals = [p for p in proposals if 'programme__proposal--category' not in p.attrs['class']]

In [303]:
len(proposals)

36

In [304]:
full_urls = ["https://en-marche.fr" + p.find('a').attrs['href'] for p in proposals]

In [305]:
url = full_urls[1]

In [306]:
r = requests.get(url)

In [307]:
text = r.text

In [308]:
text = text.replace('</br>', '')

In [309]:
soup = BeautifulSoup(text, 'html.parser')

In [310]:
article_tag = soup.find_all('article', class_='l__wrapper--slim')[0]

In [311]:
for line in article_tag.find_all(class_='arrows'):
    print(line.text) 


Nous agirons pour que les agriculteurs pèsent plus dans leurs négociations avec les industriels de l’agro-alimentaire.


Nous encouragerons le développement de véritables organisations de producteurs, avec des capacités de négociations renforcées pour peser plus dans les négociations commerciales avec les centrales d’achat de la grande distribution.


Nous organiserons un Grenelle de l’alimentation avec les représentants des agriculteurs, des industries de transformation, de la distribution et des consommateurs, afin de définir un partage équilibré de la valeur.


Nous protégerons les agriculteurs contre la volatilité des prix par la mise en place d’outils de régulation adaptés à chaque filière.


Nous proposerons des outils de gestion des risques efficaces et adaptés.


Nous permettrons aux agriculteurs de conserver les mêmes règles du jeu pour être compétitifs : favoriser la convergence sociale et fiscale au niveau européen.


Nous créerons un droit à l’erreur pour tous.


Nous donn

In [312]:
tag = article_tag.find_all(class_='arrows')[-1]

In [313]:
tag.text

'\nNous ferons confiance aux territoires pour s’organiser et trouver des solutions adaptées.\n'

In [314]:
tag.next_sibling

'\nNous encouragerons les projets alimentaires territoriaux (PAT) pour rapprocher les producteurs, les transformateurs, les distributeurs, les collectivités territoriales et atteindre 50% de produits biologiques, écologiques ou locaux dans l’ensemble de la restauration collective en 2022. \n'

In [315]:
def extract_items(url):
    r = requests.get(url)
    text = r.text.replace('</br>', '')
    soup = BeautifulSoup(text, 'html.parser')
    article_tag = soup.find_all('article', class_='l__wrapper--slim')[0]
    return [line.text.strip() for line in article_tag.find_all(class_='arrows')]

In [316]:
extract_items(full_urls[1])

['Nous agirons pour que les agriculteurs pèsent plus dans leurs négociations avec les industriels de l’agro-alimentaire.',
 'Nous encouragerons le développement de véritables organisations de producteurs, avec des capacités de négociations renforcées pour peser plus dans les négociations commerciales avec les centrales d’achat de la grande distribution.',
 'Nous organiserons un Grenelle de l’alimentation avec les représentants des agriculteurs, des industries de transformation, de la distribution et des consommateurs, afin de définir un partage équilibré de la valeur.',
 'Nous protégerons les agriculteurs contre la volatilité des prix par la mise en place d’outils de régulation adaptés à chaque filière.',
 'Nous proposerons des outils de gestion des risques efficaces et adaptés.',
 'Nous permettrons aux agriculteurs de conserver les mêmes règles du jeu pour être compétitifs : favoriser la convergence sociale et fiscale au niveau européen.',
 'Nous créerons un droit à l’erreur pour tous

On extrait toutes les propositions.

In [317]:
propositions = [extract_items(url) for url in full_urls]

In [318]:
len(propositions)

36

In [319]:
full_urls[18]

'https://en-marche.fr/emmanuel-macron/le-programme/international'

In [320]:
@interact
def print_prop(n=(0, len(propositions) - 1)):
    print(propositions[n])

In [322]:
props_sources = {}
for url, props in zip(full_urls, propositions):
    props_sources[url] = props

In [324]:
df = make_df_from_props_sources(props_sources)

In [326]:
df.head()

,proposition,source
0,Le plan d’investissement,https://en-marche.fr/emmanuel-macron/le-progra...
1,Nous instaurerons le non-cumul des mandats dan...,https://en-marche.fr/emmanuel-macron/le-progra...
2,Nous modulerons le financement des partis poli...,https://en-marche.fr/emmanuel-macron/le-progra...
3,Nous ouvrirons les postes d’autorité dans l’Et...,https://en-marche.fr/emmanuel-macron/le-progra...
4,Nous supprimerons le régime spécial de retrait...,https://en-marche.fr/emmanuel-macron/le-progra...


In [328]:
df.iloc[0, 1]

'https://en-marche.fr/emmanuel-macron/le-programme/finances-publiques'

In [325]:
df.to_csv('../projets/emmanuel_macron.csv', index=False)

# Yannick Jadot

http://avecjadot.fr/lafrancevive/

In [334]:
r = requests.get('http://avecjadot.fr/lafrancevive/')

In [335]:
soup = BeautifulSoup(r.text, 'html.parser')

In [336]:
tags = soup.find_all('div', class_='bloc-mesure')

In [337]:
links = [tag.find('a').attrs['href'] for tag in tags]

In [338]:
all([link.startswith('http://avecjadot.fr/') for link in links])

True

Extraction du titre d'une des pages.

In [339]:
link = links[0]

In [340]:
r = requests.get(link)

In [341]:
soup = BeautifulSoup(r.text, 'html.parser')

In [342]:
soup.find('div', class_='texte-mesure').text.strip().replace('\n', ' ')

'Adopter une loi de sortie progressive et définitive du nucléaire d’ici 2035, arrêter les premiers réacteurs dès 2017.'

In [343]:
def extract_data(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup.find('div', class_='texte-mesure').text.strip().replace('\n', ' ')

In [344]:
extract_data(link)

'Adopter une loi de sortie progressive et définitive du nucléaire d’ici 2035, arrêter les premiers réacteurs dès 2017.'

In [345]:
all_props = [extract_data(link) for link in links]

In [349]:
props_sources = {}
for url, props in zip(links, all_props):
    props_sources[url] = [props]

In [354]:
props_sources

{'http://avecjadot.fr/lafrancevive/1-budget-de-letat-consacre-aux-projets-choisis-citoyens-via-plateformes-civic-tech/': ['Adopter le 1% budget participatif.'],
 'http://avecjadot.fr/lafrancevive/1-million-de-jeunes-an-emploi-formation-pays-de-lunion/': ['Envoyer chaque année 1 million de jeunes en emploi ou en formation dans un autre pays de l’Union européenne.'],
 'http://avecjadot.fr/lafrancevive/2-de-dechets-2030-distribution-invendus-valorisation-compostage/': ['Aller vers une France zéro déchet en divisant par 2 notre production de déchets par habitant à l’horizon 2030. → \xa0\xa0 \xa0 Mettre en place un plan national de lutte contre le gaspillage alimentaire dans les cantines. → \xa0\xa0 \xa0 Rendre obligatoire la redistribution des invendus de la grande distribution vers les associations de solidarité, et l’élevage pour les déchets très périmés. → \xa0\xa0 \xa0 Valoriser énergétiquement les fermentescibles en généralisant la méthanisation. → \xa0\xa0 \xa0 Développer le composta

In [355]:
df = make_df_from_props_sources(props_sources)

In [356]:
df.head()

,proposition,source
0,Modifier la Constitution pour que dans l’artic...,http://avecjadot.fr/lafrancevive/constitutionn...
1,Faire de la santé une priorité nationale. → \t...,http://avecjadot.fr/lafrancevive/sante-priorit...
2,Lutter contre la souffrance animale. → \t Inte...,http://avecjadot.fr/lafrancevive/interdire-lel...
3,Renforcer la lutte contre les discriminations ...,http://avecjadot.fr/lafrancevive/renforce-la-l...
4,Favoriser la Recherche & Développement. → Mieu...,http://avecjadot.fr/lafrancevive/simplifier-pl...


In [357]:
df.to_csv('../projets/yannick_jadot.csv', index=False)

# Nicolas Dupont-Aignan 

In [26]:
r = requests.get('http://www.nda-2017.fr/themes.html')
soup = BeautifulSoup(r.text, 'html.parser')

In [27]:
len(soup.find_all('div', class_='theme'))

29

In [28]:
links = ['http://www.nda-2017.fr' + tag.find('a').attrs['href'] for tag in soup.find_all('div', class_='theme')]

In [29]:
link = links[0]

In [30]:
r = requests.get(link)

In [31]:
soup = BeautifulSoup(r.text, 'html.parser')

In [32]:
tags = soup.find_all('div', class_='proposition')

In [33]:
len(tags)

17

In [34]:
tags[0].find('a').text.strip()

'Exiger pour tout candidat à un mandat électif un casier judiciaire vierge.'

In [35]:
tags[0].find('a').attrs['href']

'#proposition32'

In [36]:
def extract_data(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')
    tags = soup.find_all('div', class_='proposition')
    return [tag.find('a').text.strip() for tag in tags]

In [37]:
all_props = [extract_data(link) for link in links]

In [43]:
len(all_props)

29

In [47]:
props_sources = {}
for url, props in zip(links, all_props):
    props_sources[url] = props

In [48]:
df = make_df_from_props_sources(props_sources)

In [49]:
df

,proposition,source
0,Cibler et simplifier les 20 milliards d’euros ...,http://www.nda-2017.fr/theme/economie-travail
1,Exonérer de charges pendant 5 ans le recruteme...,http://www.nda-2017.fr/theme/economie-travail
2,Relancer la « Participation gaullienne » des s...,http://www.nda-2017.fr/theme/economie-travail
3,Parvenir à l’égalité Femmes/Hommes au travail ...,http://www.nda-2017.fr/theme/economie-travail
4,Abroger la directive dite « travailleurs détac...,http://www.nda-2017.fr/theme/economie-travail
5,Adopter un « BUY French ACT » à la manière des...,http://www.nda-2017.fr/theme/economie-travail
6,Garantir des débouchés plus nombreux aux PME f...,http://www.nda-2017.fr/theme/economie-travail
7,Baisser de moitié le taux de l’Impôt sur les S...,http://www.nda-2017.fr/theme/economie-travail
8,Mettre en place un étiquetage obligatoire sur ...,http://www.nda-2017.fr/theme/economie-travail
9,Accorder aux produits 100% fabriqués en France...,http://www.nda-2017.fr/theme/economie-travail


In [50]:
df.to_csv('../projets/nicolas_dupont-aignan.csv', index=False)

# Nathalie Arthaud 

http://www.nathalie-arthaud.info/

# François Asselineau 

https://www.upr.fr/programme-elections-presidentielles-france